# 2. Wrapping the RAG Agent into an ACP Server

Let's wrap the RAG CrewAI agent we created in the previously in ACP server and then run the ACP server to activate the agent so it can be discoverable by an ACP client.

## 2.1. Wrap the Agent in ACP  Server

We will now take the same code we worked on in `1-CrewAI-RAG-Agent` notebook and wrap it in a python file called: `crew_agent_server`.

To make the agent ACP compliant, we can use the `@server.agent()` decorator to define our agent. The name is inferred from the function name, and the description is pulled from the docstring. Here's the minimal structure needed for an ACP-compliant agent:

```python
@server.agent()
async def travel_package_agent(input: list[Message]) -> AsyncGenerator[RunYield, RunYieldResume]:
    "This is an agent for questions around policy coverage, it uses a RAG pattern to find answers based on policy documentation. Use it to help answer questions on coverage and waiting periods."
    # Here goes the function definition
    # ....
    task_output = ...
    yield Message(parts=[MessagePart(content=str(task_output))])
```
This configuration establishes several critical aspects of the agent:
- **Function Definition**: The core functionality that determines what the agent does;
- **Input Parameter**: The input parameter accepts a list of Message objects; 
- **Return Type**: The AsyncGenerator[RunYield, RunYieldResume] return type enables both streaming responses and the await pattern:
   - AsyncGenerator: An async generator object that can be iterated with async for and supports await operations
   - RunYield: The type of values this generator yields (sends out)
   - RunYieldResume: The type of values this generator receives when resumed (sent back in) (in the definition below, we will only use RunYield)
- **Documentation**: The docstring provides a human-readable description of the agent

Run the following cell to copy the content of the cell to the file `crew_agent_server.py` which will be saved under the folder `travel_acp_project`. 

In [ ]:
%%writefile ../travel_acp_project/src/crew_agent_server.py
from collections.abc import AsyncGenerator
from acp_sdk.models import Message, MessagePart
from acp_sdk.server import RunYield, RunYieldResume, Server

from crewai import Crew, Task, Agent, LLM
from crewai_tools import RagTool

import nest_asyncio
nest_asyncio.apply()

server = Server()
llm = LLM(model="openai/gpt-4", max_tokens=1024)

config = {
    "llm": {
        "provider": "openai",
        "config": {
            "model": "gpt-4",
        }
    },
    "embedding_model": {
        "provider": "openai",
        "config": {
            "model": "text-embedding-ada-002"
        }
    }
}
rag_tool = RagTool(config=config,  
                   chunk_size=1200,       
                   chunk_overlap=200,     
                  )

rag_tool.add("../data/Europe-Packages-from-Ahmedabad.pdf", data_type="pdf_file")
rag_tool.add("../data/Europe-Packages-from-Chennai.pdf", data_type="pdf_file")
rag_tool.add("../data/Europe-Packages-from-Delhi.pdf", data_type="pdf_file")
rag_tool.add("../data/Europe-Packages-from-Hyderabad.pdf", data_type="pdf_file")


@server.agent()
async def travel_package_agent(input: list[Message]) -> AsyncGenerator[RunYield, RunYieldResume]:
    "This is an agent for questions around policy coverage, it uses a RAG pattern to find answers based on policy documentation. Use it to help answer questions on coverage and waiting periods."

    travel_package_agent = Agent(
        role="Senior Travel Package Advisor",
        goal="Accurately answer questions about Europe travel packages including pricing, itinerary, inclusions, exclusions, and departure cities.",
        backstory=(
            "You are a seasoned travel consultant specializing in curated Europe travel packages. "
            "You assist customers by interpreting and explaining offerings from detailed travel brochures "
            "for different departure cities like Ahmedabad, Delhi, Chennai, and Hyderabad."
        ),
        verbose=True,
        allow_delegation=False,
        llm=llm,
        tools=[rag_tool],
        max_retry_limit=5
    )
    
    task1 = Task(
         description=input[0].parts[0].content,
         expected_output = "A comprehensive response as to the users question",
         agent=travel_package_agent
    )
    crew = Crew(agents=[travel_package_agent], tasks=[task1], verbose=True)
    
    task_output = await crew.kickoff_async()
    yield Message(parts=[MessagePart(content=str(task_output))])

if __name__ == "__main__":
    server.run(port=8001)

## 2.2. Run the Travel Package ACP Server

**Note: How to set up `travel_acp_project` locally on our machine using the `uv` tool?**

- First install `uv` by checking this [link](https://docs.astral.sh/uv/getting-started/installation/).

After that, in the terminal we can type the following commands:
- `cd travel_acp_project`: navigate to directory where we created `travel_acp_project`.
- `uv venv`: to create a virtual environment
- `uv sync`: to install the dependencies defined in `pyproject.toml` file.
- `.venv\Scripts\activate`: to activate the virtual environment.

We can then run the server using `uv run`.  Since this code uses an OpenAI model, we would also need to specify an openAI API key in a `.env` file like this: `OPENAI_API_KEY=sk-...`.

Now to activate our configured ACP agent, we would need to run our agent server. The folder `travel_acp_project` has been set up for us so we can run the agent server using `uv`:

- Type `uv run python src/crew_agent_server.py` to run the server and activate our ACP agent.

## 2.3. Calling an Travel Package ACP Agent using the Client

Now we will create an ACP client to interact with the ACP server. 

**Why `nest_asyncio` is needed?**

We will run the ACP client from the environment of the jupyter notebook. Since the ACP client is an an asyncio process (can send asynchronous requests to the server), we will need first to import `nest_asyncio`. This is because the Jupyter notebook already runs an asyncio event loop in the background to handle various operations. When we try to run our own `asyncio` code (`asyncio.run()`), Python throws an error because we're trying to start a new event loop while one is already running. `nest_asyncio.apply()` allows nested event loops, which lets us run async code directly in Jupyter cells.

In [9]:
import nest_asyncio
nest_asyncio.apply()

Since the agent is running locally at port 8001, all we need to do is to pass the endpoint `http://localhost:8001` to the client so it can discover the agents hosted inside the server. Using the `client`, we can execute the agent synchronously using the method `run_sync` (Synchronous execution waits until the agent completes processing and returns a single response). We will need to pass to `run_sync` the name of agent and the input message.

In [10]:
from acp_sdk.client import Client
import asyncio
from colorama import Fore

async def example() -> None:
    async with Client(base_url="http://localhost:8001") as client:
        run = await client.run_sync(
            # agent="travel_package_agent", input="What is the total cost and duration of the 'Best of Switzerland & Italy' package from Hyderabad?"
            agent="travel_package_agent", input="What is the best time to go for Europe trip from Hyderabad?"
        )
        print(Fore.YELLOW + run.output[0].parts[0].content + Fore.RESET)

In [11]:
asyncio.run(example())

You can plan your Europe trip from Hyderabad at any time of the year as each season has its unique appeal. 

1. Peak Season (June to August): These are the summer months with temperatures ranging from 15°C to 30°C. Destinations like Santorini are at their best, and outdoor activities in France, Italy, and Spain are popular. Music festivals, long daylight hours, and vibrant city life attract large crowds to iconic landmarks.

2. Waning Season (December to February):  The winter brings temperatures between -5°C to 10°C, which is ideal for skiing in the Swiss Alps or witnessing the Northern Lights in Norway. Festive Christmas markets in cities like Vienna and Prague provide magical experiences with fewer tourists and a cosy winter charm.

Reaching Europe from Hyderabad is convenient with several flight options from Rajiv Gandhi International Airport to major European airports like Heathrow in London, or Frankfurt in Germany. Airlines such as Lufthansa, Emirates, Etihad, Qatar Airways, and